In [1]:
from imutils import paths
import numpy as np
import imutils
import pickle
import cv2
import os

detector = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")

imagePaths = list(paths.list_images("dataset"))
knownEmbeddings = [] #feature vector 128-d
knownNames = [] #Names of detected person
total = 0 #total faces processed

for (i, imagePath) in enumerate(imagePaths):
    name=imagePath.split(os.path.sep)[-2]#names
    image=cv2.imread(imagePath)
    image=imutils.resize(image, width=600)
    (h,w)=image.shape[:2]
    imageBlob=cv2.dnn.blobFromImage(cv2.resize(image,(300,300)),1.0,(300,300),(104.0,177.0,123.0))
    detector.setInput(imageBlob)
    detections=detector.forward()
    #[,frame,no. of detections,[class_id,class_score,conf,x,y,h,w]]
    i=np.argmax(detections[0,0,:,2])
    confidence=detections[0, 0, i, 2]
    if confidence>0.5:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        face = image[startY:endY, startX:endX]
        (fH, fW) = face.shape[:2]
        faceBlob = cv2.dnn.blobFromImage(face,1.0/255,(96,96),(0,0,0),swapRB=True,crop=False)
        embedder.setInput(faceBlob)
        vec = embedder.forward()
        knownNames.append(name)
        knownEmbeddings.append(vec.flatten())
        total += 1
data = {"embeddings": knownEmbeddings, "names": knownNames}
#print(data)
f = open("embeddings.pickle", "wb")
f.write(pickle.dumps(data))
f.close()

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import argparse
import pickle

data = pickle.loads(open("embeddings.pickle", "rb").read())

le = LabelEncoder()
labels = le.fit_transform(data["names"])

recognizer = SVC(kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)

f = open("recognizer.pickle", "wb")
f.write(pickle.dumps(recognizer))
f.close()
# write the label encoder to disk
f = open("le.pickle", "wb")
f.write(pickle.dumps(le))
f.close()

In [7]:
import numpy as np
import imutils
import pickle
import cv2
import os

detector = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')
embedder = cv2.dnn.readNetFromTorch('openface_nn4.small2.v1.t7')

recognizer = pickle.loads(open('recognizer.pickle', "rb").read())
le = pickle.loads(open('le.pickle', "rb").read())
path=os.getcwd()+"\\test3.jpg"
image=cv2.imread(path)
image = imutils.resize(image, width=600)
(h, w) = image.shape[:2]
imageBlob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300),(104.0, 177.0, 123.0), swapRB=False, crop=False)

detector.setInput(imageBlob)
detections = detector.forward()
#[,frame,no. of detections,[class_id,class_score,conf,x,y,h,w]]

for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])#3:7 gives us the bounding box coord.
        (startX,startY,endX,endY)=box.astype("int")
        face=image[startY:endY,startX:endX]#getting face
        (fH, fW)=face.shape[:2]
        if fW<20 or fH<20:
            continue
        faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),(0, 0, 0), swapRB=True, crop=False)
        embedder.setInput(faceBlob)
        vec = embedder.forward()
        # perform classification to recognize the face
        preds = recognizer.predict_proba(vec)[0]#(3,)
        #print(recognizer.predict_proba(vec))
        j = np.argmax(preds)
        proba = preds[j]
        name = le.classes_[j]
        text = str(name)+" "+str(format(proba*100,".1f"))
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY),(0, 0, 255), 2)
        cv2.putText(image, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
cv2.imshow("Image", image)
cv2.waitKey(0)

-1